In [33]:
from dataclasses import dataclass

@dataclass
class Packet:
    data: str
    parity_error: bool
    computed_crc: str
    crc_error: bool
    time: datetime.datetime

In [34]:
help(datetime.datetime)

Help on class datetime in module datetime:

class datetime(date)
 |  datetime(year, month, day[, hour[, minute[, second[, microsecond[,tzinfo]]]]])
 |  
 |  The year, month and day arguments are required. tzinfo may be None, or an
 |  instance of a tzinfo subclass. The remaining arguments may be ints.
 |  
 |  Method resolution order:
 |      datetime
 |      date
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __radd__(self, value

In [44]:
import re

packets = []
with open("../MonitorCaptures/capture2_noerrors_fixedtrim_08292021_12_08am.txt", "r") as f:
    for line in f:
        rslt = re.search(r"([0-9]{2})?:?([0-9]{2})?:?([0-9]{2})?\.?([0-9]{3})?\s?-?>?\s?R [0-9]\.[0-9]{3}: ([0-9A-F]+)(-PE:)? CRC=([0-9A-F]{2})( CRC error)?", line)
        if rslt:
            hh = 0
            mm = 0
            ss = 0
            mil = 0
            if rslt.groups()[0]: hh = int(rslt.groups()[0])
            if rslt.groups()[1]: mm = int(rslt.groups()[1])
            if rslt.groups()[2]: ss = int(rslt.groups()[2])
            if rslt.groups()[3]: mil = int(rslt.groups()[3])
            packet = Packet(data=rslt.groups()[4], parity_error=(rslt.groups()[5] != None), computed_crc=rslt.groups()[6], crc_error=(rslt.groups()[7] != None), time=datetime.datetime(1,1,1,hh,mm,ss,mil*1000))
            packets.append(packet)

In [46]:
packets[-1]

Packet(data='40001001006202190611003A0000004000050000010007', parity_error=False, computed_crc='AE', crc_error=False, time=datetime.datetime(1, 1, 1, 0, 8, 7, 465000))

In [8]:
from collections import Counter
packet_freqs = Counter([x.data for x in packets])

In [129]:
all_decoded_lines = Counter([*decode_packet_data(x.data) for x in packets])

SyntaxError: iterable unpacking cannot be used in comprehension (18069309.py, line 1)

In [138]:
list(all_decoded_lines.keys())

['src = 0x00tgt = 0x00type = 0x10Thermostat Settings',
 'src = 0x00tgt = 0x00type = 0x10HeatingOn1: 01',
 'src = 0x00tgt = 0x00type = 0x10GasMode: 62',
 'src = 0x00tgt = 0x00type = 0x10DHWon: 02',
 'src = 0x00tgt = 0x00type = 0x10temp? = 25.6C',
 'src = 0x00tgt = 0x00type = 0x10[i=8] UNK: 11',
 'src = 0x00tgt = 0x00type = 0x10[i=9] UNK: 00',
 'src = 0x00tgt = 0x00type = 0x10RoomTarget1: 02',
 'src = 0x00tgt = 0x00type = 0x10[i=11] UNK: 00',
 'src = 0x00tgt = 0x00type = 0x10Flags_Including_QuietMode1_at_pos2: 00',
 'src = 0x00tgt = 0x00type = 0x10Flags_Including_QuietMode1_at_pos2: 40',
 'src = 0x00tgt = 0x00type = 0x10[i=14] UNK: 00',
 'src = 0x00tgt = 0x00type = 0x10[i=15] UNK: 00',
 'src = 0x00tgt = 0x00type = 0x10[i=16] UNK: 27',
 'src = 0x00tgt = 0x00type = 0x10[i=17] UNK: 40',
 'src = 0x00tgt = 0x00type = 0x10[i=18] UNK: 01',
 'src = 0x00tgt = 0x00type = 0x10[i=19] UNK: 00',
 'src = 0x00tgt = 0x00type = 0x10Flags ([DHWactive] ? ? ? ? [DHWboosterActive] ?): 0b0',
 'src = 0x00tgt = 

In [60]:
import datetime
def video_time_to_terminal_time(video_time):
    video_cal_timestamp = datetime.datetime(1,1,1, 0, 0, 4, 723000)
    terminal_cal_timestamp = datetime.datetime(1,1,1, 0, 6, 5, 116000)
    return ((terminal_cal_timestamp - video_cal_timestamp) + video_time)
    

def terminal_time_to_video_time(terminal_time):
    video_cal_timestamp = datetime.datetime(1,1,1, 0, 0, 4, 723000)
    terminal_cal_timestamp = datetime.datetime(1,1,1, 0, 6, 5, 116000)
    return ((video_cal_timestamp - terminal_cal_timestamp) + terminal_time)

In [267]:
len('8000100160001906110002000040000027000000000000')

46

In [286]:
def decode_packet_data(data: str):
    d = bytes.fromhex(data)
    src = d[0]
    tgt = d[1]
    pkt_type = d[2]
    txt = []
    txt.append(f"src = 0x{'%02x'%src}")
    txt.append(f"tgt = 0x{'%02x'%tgt}")
    txt.append(f"type = 0x{'%02x'%pkt_type}")
    if pkt_type == 0x10 and src == 0:
        txt.append("Thermostat Settings (Thermostat -> Pump)")
        for i in range(3, 23):
            if i == 3: txt.append(f"HeatingOn1: {'%02x'%d[i]}") # always 1
            elif i == 4: txt.append(f"Gasmode Flags: {bin(d[i])}") # seen 0b1100010 (895) and 0b1100000 (6)
            elif i == 5: txt.append(f"Flags: (? [DHWon]): {'%02x'%d[i]}") # seen 0b10 (895) and 0b0 (6)
            elif i == 6 or i == 7:
                if i == 6: txt.append(f"expected temp? = {d[i]}.{d[i+1]}C") # seen 25.6C (866); also 24.5 (9), 25.0 (11), 26.1 (15); translates to 78.08F (typ); also 76.1, 77.0, 78.98~==79
            elif i == 8: txt.append(f"Flags: ?: {bin(d[i])}") # seen 0b0010001 (888), 0b0110001 (9), 0b1010001 (4)
            elif i == 10: txt.append(f"RoomTarget1: {'%02x'%d[i]}") # always 2
            elif i == 12: txt.append(f"i=12Flags ([QuietMode1] ? ?): {bin(d[i])}") # always 0
            elif i == 13: txt.append(f"i=13Flags ([QuietMode1] ? ?): {bin(d[i])}") # always 0b1000000
            elif i == 16: txt.append(f"Flags: ?: {bin(d[i])}") # seen 0b100111 (797), 0b000111 (10), 0b000101 (94)
            elif i == 20: txt.append(f"Flags ([DHWactive] ? ? ? ? [DHWboosterActive] ?): {bin(d[i])}") # always 0
            elif i == 21: txt.append(f"DHWComfort1: {'%02x'%d[i]}") # always 0
            else: txt.append(f"[i={i}] UNK: {'%02x'%d[i]}")
    if pkt_type == 0x10 and src == 0x40:
        txt.append("Thermostat Settings (Pump -> Thermostat)")
        for i in range(3, 23):
            if i == 3: txt.append(f"HeatingOn2: {'%02x'%d[i]}")
            elif i == 5: txt.append(f"Flags ([DHWtankValve] [AddZoneValve] [MainZoneValve] ? ? ? [CoolingValve] [HeatingValve]): {bin(d[i])}")
            elif i == 6: txt.append(f"Flags ([SHCtank] ? ? ? [3WayValve]): {bin(d[i])}") # 25.6C (866), 26.1 (15), 25.0 (11), 24.5 (9)
            elif i == 7 or i == 8:
                if i == 7: txt.append(f"expected temp?: {d[i]}.{d[i+1]}C")
            elif i == 11: txt.append(f"RoomTarget2: {'%02x'%d[i]}")
            elif i == 14: txt.append(f"Flags ([QuietMode2] ? ?): {bin(d[i])}")
            elif i == 21: txt.append(f"Flags ([CircPump] ? ? [Compressor]): {bin(d[i])}")
            elif i == 22: txt.append(f"Flags ([DHWmode] ? ?): {bin(d[i])}")
            else: txt.append(f"[i={i}] UNK: {'%02x'%d[i]}")
    if pkt_type == 0x10 and src == 0x80:
        txt.append("Thermostat settings? (??? -> Thermostat)")
        # 8000100160001906110002000040000027000000000000
        for i in range(3, 23):
            txt.append(f"[i={i}] UNK: {'%02x'%d[i]}")
    if pkt_type == 0x15 and src == 0x00:
        txt.append("Heartbeat (Thermostat -> Pump)")
    if pkt_type == 0xFF and src == 0x40:
        txt.append("ACK (Pump -> Thermostat)")
    if pkt_type == 0x12 and src == 0x00:
        # Time?
        txt.append("Time from Thermostat?")
        for i in range(3, 18):
            if i == 7: txt.append(f"Time1?: {d[i]}")
            elif i == 8: txt.append(f"Time2?: {d[i]}")
            elif i == 9: txt.append(f"Time3?: {d[i]}")
            else: txt.append(f"[i={i}] UNK: {'%02x'%d[i]}")
    if pkt_type == 0x12 and src == 0x40:
        # Time response?
        txt.append("Time response from heat pump?")
        for i in range(3, 10):
            txt.append(f"[i={i}] UNK: {'%02x'%d[i]}")
    if pkt_type == 0x11 and src == 0x00:
        txt.append("Room Temperature (Thermostat)")
        txt.append(f"{d[8]+(d[9]/256)}")
    if pkt_type == 0x1F and src == 0x00:
        # 00001F
        txt.append("Unknown Request (Thermostat)")
    if pkt_type == 0x1F and src == 0x40:
        # 40001F0000000000000000000000000000000000000007
        txt.append("Unknown Response (Thermostat)")
    if pkt_type == 0x11 and src == 0x40:
        # 400011000000000018A0000000000AE4
        for i in range(3, 16):
            if i == 8 or i == 9:
                if i == 8: txt.append(f"Outdoor Temperature (TempOut1): {d[i]+(d[i+1]/256)}C")
            if i == 14 or i == 15:
                if i == 14: txt.append(f"Refrigerant Temperature?: {d[i]+(d[i+1]/256)}C")
            #else: txt.append(f"[i={i}] UNK: {'%02x'%d[i]}")
    if pkt_type == 0x20 and src == 0x00:
        txt.append("Device identification request? (Thermostat)")
        for i in range(3, 4):
            txt.append(f"[i={i}] UNK: {'%02x'%d[i]}")
    if data == "00F0300023B00020111F100193006CC1470D00520D0302":
        txt.append("Unknown message type 0x30 from Thermostat (to external controller?) -- need more context")
    if data == "4000200223B00020102010250193406CC1470D00520D03":
        txt.append("Device identification response? 0x20 from heat pump -- need more context")
    if data == "0000A100000000000000000000000000000000":
        txt.append("Unknown message type 0xA1 from thermostat -- need more context")
    if data == "4000A100000000000000000000000000000000":
        txt.append("Unknown message type 0xA1 from heat pump -- need more context")
    if data == "0000B1000000000000000000000000000000000000":
        txt.append("Unknown message type 0xB1 from thermostat -- need more context")
    if data == "4000B100464251323450564A550000000000000000":
        txt.append("Unknown message type 0xB1 from heat pump -- need more context")
    if data == "8000180100000000000000":
        txt.append("Unknown message type 0x18 from 0x80 -- need more context")
    if data == "808000":
        txt.append("Unknown message type 0x00 from 0x80 -- need more context")
        
        
        
    # Unknown (meaningful?) payload: 00F0300023B00020111F100193006CC1470D00520D0302
        
    # Unknown meaningful payload: 8000100160001906110002000040000027000000000000
    # Unknown meaningful payload: 8000100162021906110002000040000007000000000000
    # 40001200000080000000: Clock ACK from heat pump?
    # 000012000000002B1D000100000000000000: Time from thermostat?
    
    return txt

def generate_decoded_lines(packets):
    all_decoded_lines = []
    for packet in packets:
        decoded = decode_packet_data(packet.data)
        prefix = f"{decoded[0]}{decoded[1]}{decoded[2]}"
        for i in range(3, len(decoded)):
            all_decoded_lines.append(prefix+decoded[i])
    all_decoded_lines = Counter(all_decoded_lines)
    return all_decoded_lines
all_decoded_lines = generate_decoded_lines(packets)

def add_line_counts(decoded_packet_txt):
    prefix = f"{decoded_packet_txt[0]}{decoded_packet_txt[1]}{decoded_packet_txt[2]}"
    for i in range(3, len(decoded_packet_txt)):
        decoded_packet_txt[i] += f"; Count = {all_decoded_lines[prefix+decoded_packet_txt[i]]}"
    return decoded_packet_txt

In [289]:
for packet in packets:
    decoded = decode_packet_data(packet.data)
    if True:
    #if len(decoded) == 3:
    #if packet.data.startswith("400011"):
    #if decoded[0] != "src = 0x80" and decoded[2] != "type = 0x10" and decoded[2] != "type = 0xff" and decoded[2] != "type = 0x15":
    #if decoded[0] != "src = 0x80":# and decoded[2] != "type = 0x18" and decoded[2] != "type = 0x00":
        print(packet.data)
        print(f"\tCount = {packet_freqs[packet.data]}")
        print(f"\tTime = {packet.time.hour}:{packet.time.minute}:{packet.time.second}.{packet.time.microsecond/1000}")
    #    decoded_lines = decode_packet_data(packet.data)
        print('\t'+'\n\t'.join(add_line_counts(decoded)))

000015100000F0FFFF00F0FFFF00F0FFFF00F0FFFF
	Count = 226
	Time = 0:0:0.0.0
	src = 0x00
	tgt = 0x00
	type = 0x15
	Heartbeat (Thermostat -> Pump); Count = 226
4000FF
	Count = 226
	Time = 0:0:0.0.0
	src = 0x40
	tgt = 0x00
	type = 0xff
	ACK (Pump -> Thermostat); Count = 226
0000100162021906110002000040000027400100000000
	Count = 760
	Time = 0:0:0.0.0
	src = 0x00
	tgt = 0x00
	type = 0x10
	Thermostat Settings (Thermostat -> Pump); Count = 901
	HeatingOn1: 01; Count = 901
	Gasmode Flags: 0b1100010; Count = 895
	Flags: (? [DHWon]): 02; Count = 895
	expected temp? = 25.6C; Count = 866
	Flags: ?: 0b10001; Count = 888
	[i=9] UNK: 00; Count = 901
	RoomTarget1: 02; Count = 901
	[i=11] UNK: 00; Count = 901
	i=12Flags ([QuietMode1] ? ?): 0b0; Count = 901
	i=13Flags ([QuietMode1] ? ?): 0b1000000; Count = 901
	[i=14] UNK: 00; Count = 901
	[i=15] UNK: 00; Count = 901
	Flags: ?: 0b100111; Count = 797
	[i=17] UNK: 40; Count = 901
	[i=18] UNK: 01; Count = 901
	[i=19] UNK: 00; Count = 901
	Flags ([DHWactive]

	Count = 38
	Time = 0:0:0.0.0
	src = 0x40
	tgt = 0x00
	type = 0x20
	Device identification response? 0x20 from heat pump -- need more context; Count = 38
0000100162021906110002000040000027400100000000
	Count = 760
	Time = 0:0:0.0.0
	src = 0x00
	tgt = 0x00
	type = 0x10
	Thermostat Settings (Thermostat -> Pump); Count = 901
	HeatingOn1: 01; Count = 901
	Gasmode Flags: 0b1100010; Count = 895
	Flags: (? [DHWon]): 02; Count = 895
	expected temp? = 25.6C; Count = 866
	Flags: ?: 0b10001; Count = 888
	[i=9] UNK: 00; Count = 901
	RoomTarget1: 02; Count = 901
	[i=11] UNK: 00; Count = 901
	i=12Flags ([QuietMode1] ? ?): 0b0; Count = 901
	i=13Flags ([QuietMode1] ? ?): 0b1000000; Count = 901
	[i=14] UNK: 00; Count = 901
	[i=15] UNK: 00; Count = 901
	Flags: ?: 0b100111; Count = 797
	[i=17] UNK: 40; Count = 901
	[i=18] UNK: 01; Count = 901
	[i=19] UNK: 00; Count = 901
	Flags ([DHWactive] ? ? ? ? [DHWboosterActive] ?): 0b0; Count = 901
	DHWComfort1: 00; Count = 901
	[i=22] UNK: 00; Count = 901
400010010

	Time = 0:0:0.0.0
	src = 0x00
	tgt = 0x03
	type = 0x20
	Device identification request? (Thermostat); Count = 38
	[i=3] UNK: 34; Count = 38
0000100162021906110002000040000027400100000000
	Count = 760
	Time = 0:0:0.0.0
	src = 0x00
	tgt = 0x00
	type = 0x10
	Thermostat Settings (Thermostat -> Pump); Count = 901
	HeatingOn1: 01; Count = 901
	Gasmode Flags: 0b1100010; Count = 895
	Flags: (? [DHWon]): 02; Count = 895
	expected temp? = 25.6C; Count = 866
	Flags: ?: 0b10001; Count = 888
	[i=9] UNK: 00; Count = 901
	RoomTarget1: 02; Count = 901
	[i=11] UNK: 00; Count = 901
	i=12Flags ([QuietMode1] ? ?): 0b0; Count = 901
	i=13Flags ([QuietMode1] ? ?): 0b1000000; Count = 901
	[i=14] UNK: 00; Count = 901
	[i=15] UNK: 00; Count = 901
	Flags: ?: 0b100111; Count = 797
	[i=17] UNK: 40; Count = 901
	[i=18] UNK: 01; Count = 901
	[i=19] UNK: 00; Count = 901
	Flags ([DHWactive] ? ? ? ? [DHWboosterActive] ?): 0b0; Count = 901
	DHWComfort1: 00; Count = 901
	[i=22] UNK: 00; Count = 901
40001001006202190611003

	Count = 168
	Time = 0:0:0.0.0
	src = 0x00
	tgt = 0x00
	type = 0x12
	Time from Thermostat?; Count = 225
	[i=3] UNK: 00; Count = 225
	[i=4] UNK: 00; Count = 225
	[i=5] UNK: 00; Count = 225
	[i=6] UNK: 00; Count = 225
	Time1?: 43; Count = 225
	Time2?: 29; Count = 168
	Time3?: 0; Count = 168
	[i=10] UNK: 01; Count = 225
	[i=11] UNK: 00; Count = 225
	[i=12] UNK: 00; Count = 225
	[i=13] UNK: 00; Count = 225
	[i=14] UNK: 00; Count = 225
	[i=15] UNK: 00; Count = 225
	[i=16] UNK: 00; Count = 225
	[i=17] UNK: 00; Count = 225
40001200000080000000
	Count = 225
	Time = 0:0:0.0.0
	src = 0x40
	tgt = 0x00
	type = 0x12
	Time response from heat pump?; Count = 225
	[i=3] UNK: 00; Count = 225
	[i=4] UNK: 00; Count = 225
	[i=5] UNK: 00; Count = 225
	[i=6] UNK: 80; Count = 225
	[i=7] UNK: 00; Count = 225
	[i=8] UNK: 00; Count = 225
	[i=9] UNK: 00; Count = 225
00062034
	Count = 37
	Time = 0:0:0.0.0
	src = 0x00
	tgt = 0x06
	type = 0x20
	Device identification request? (Thermostat); Count = 37
	[i=3] UNK: 34; 

	Count = 225
	Time = 0:0:0.0.0
	src = 0x40
	tgt = 0x00
	type = 0x12
	Time response from heat pump?; Count = 225
	[i=3] UNK: 00; Count = 225
	[i=4] UNK: 00; Count = 225
	[i=5] UNK: 00; Count = 225
	[i=6] UNK: 80; Count = 225
	[i=7] UNK: 00; Count = 225
	[i=8] UNK: 00; Count = 225
	[i=9] UNK: 00; Count = 225
00092034
	Count = 37
	Time = 0:0:0.0.0
	src = 0x00
	tgt = 0x09
	type = 0x20
	Device identification request? (Thermostat); Count = 37
	[i=3] UNK: 34; Count = 37
0000100162021906110002000040000027400100000000
	Count = 760
	Time = 0:0:0.0.0
	src = 0x00
	tgt = 0x00
	type = 0x10
	Thermostat Settings (Thermostat -> Pump); Count = 901
	HeatingOn1: 01; Count = 901
	Gasmode Flags: 0b1100010; Count = 895
	Flags: (? [DHWon]): 02; Count = 895
	expected temp? = 25.6C; Count = 866
	Flags: ?: 0b10001; Count = 888
	[i=9] UNK: 00; Count = 901
	RoomTarget1: 02; Count = 901
	[i=11] UNK: 00; Count = 901
	i=12Flags ([QuietMode1] ? ?): 0b0; Count = 901
	i=13Flags ([QuietMode1] ? ?): 0b1000000; Count = 901

...

	Flags ([DHWmode] ? ?): 0b111; Count = 901
00012034
	Count = 38
	Time = 0:0:0.0.0
	src = 0x00
	tgt = 0x01
	type = 0x20
	Device identification request? (Thermostat); Count = 38
	[i=3] UNK: 34; Count = 38
0000100162021906110002000040000027400100000000
	Count = 760
	Time = 0:0:0.0.0
	src = 0x00
	tgt = 0x00
	type = 0x10
	Thermostat Settings (Thermostat -> Pump); Count = 901
	HeatingOn1: 01; Count = 901
	Gasmode Flags: 0b1100010; Count = 895
	Flags: (? [DHWon]): 02; Count = 895
	expected temp? = 25.6C; Count = 866
	Flags: ?: 0b10001; Count = 888
	[i=9] UNK: 00; Count = 901
	RoomTarget1: 02; Count = 901
	[i=11] UNK: 00; Count = 901
	i=12Flags ([QuietMode1] ? ?): 0b0; Count = 901
	i=13Flags ([QuietMode1] ? ?): 0b1000000; Count = 901
	[i=14] UNK: 00; Count = 901
	[i=15] UNK: 00; Count = 901
	Flags: ?: 0b100111; Count = 797
	[i=17] UNK: 40; Count = 901
	[i=18] UNK: 01; Count = 901
	[i=19] UNK: 00; Count = 901
	Flags ([DHWactive] ? ? ? ? [DHWboosterActive] ?): 0b0; Count = 901
	DHWComfort1: 00; 

	src = 0x00
	tgt = 0x04
	type = 0x20
	Device identification request? (Thermostat); Count = 38
	[i=3] UNK: 34; Count = 38
0000100162021906110002000040000027400100000000
	Count = 760
	Time = 0:4:49.832.0
	src = 0x00
	tgt = 0x00
	type = 0x10
	Thermostat Settings (Thermostat -> Pump); Count = 901
	HeatingOn1: 01; Count = 901
	Gasmode Flags: 0b1100010; Count = 895
	Flags: (? [DHWon]): 02; Count = 895
	expected temp? = 25.6C; Count = 866
	Flags: ?: 0b10001; Count = 888
	[i=9] UNK: 00; Count = 901
	RoomTarget1: 02; Count = 901
	[i=11] UNK: 00; Count = 901
	i=12Flags ([QuietMode1] ? ?): 0b0; Count = 901
	i=13Flags ([QuietMode1] ? ?): 0b1000000; Count = 901
	[i=14] UNK: 00; Count = 901
	[i=15] UNK: 00; Count = 901
	Flags: ?: 0b100111; Count = 797
	[i=17] UNK: 40; Count = 901
	[i=18] UNK: 01; Count = 901
	[i=19] UNK: 00; Count = 901
	Flags ([DHWactive] ? ? ? ? [DHWboosterActive] ?): 0b0; Count = 901
	DHWComfort1: 00; Count = 901
	[i=22] UNK: 00; Count = 901
40001001006202190611003A00000040002700

	Time = 0:5:41.271.0
	src = 0x00
	tgt = 0x00
	type = 0x10
	Thermostat Settings (Thermostat -> Pump); Count = 901
	HeatingOn1: 01; Count = 901
	Gasmode Flags: 0b1100010; Count = 895
	Flags: (? [DHWon]): 02; Count = 895
	expected temp? = 25.6C; Count = 866
	Flags: ?: 0b10001; Count = 888
	[i=9] UNK: 00; Count = 901
	RoomTarget1: 02; Count = 901
	[i=11] UNK: 00; Count = 901
	i=12Flags ([QuietMode1] ? ?): 0b0; Count = 901
	i=13Flags ([QuietMode1] ? ?): 0b1000000; Count = 901
	[i=14] UNK: 00; Count = 901
	[i=15] UNK: 00; Count = 901
	Flags: ?: 0b100111; Count = 797
	[i=17] UNK: 40; Count = 901
	[i=18] UNK: 01; Count = 901
	[i=19] UNK: 00; Count = 901
	Flags ([DHWactive] ? ? ? ? [DHWboosterActive] ?): 0b0; Count = 901
	DHWComfort1: 00; Count = 901
	[i=22] UNK: 00; Count = 901
40001001006202190611003A0000004000270000010007
	Count = 760
	Time = 0:5:41.340.0
	src = 0x40
	tgt = 0x00
	type = 0x10
	Thermostat Settings (Pump -> Thermostat); Count = 901
	HeatingOn2: 01; Count = 901
	[i=4] UNK: 00; C

	Time = 0:6:33.116.0
	src = 0x40
	tgt = 0x00
	type = 0x10
	Thermostat Settings (Pump -> Thermostat); Count = 901
	HeatingOn2: 01; Count = 901
	[i=4] UNK: 00; Count = 901
	Flags ([DHWtankValve] [AddZoneValve] [MainZoneValve] ? ? ? [CoolingValve] [HeatingValve]): 0b1100010; Count = 895
	Flags ([SHCtank] ? ? ? [3WayValve]): 0b10; Count = 895
	expected temp?: 25.0C; Count = 11
	[i=9] UNK: 11; Count = 888
	[i=10] UNK: 00; Count = 901
	RoomTarget2: 3a; Count = 901
	[i=12] UNK: 00; Count = 901
	[i=13] UNK: 00; Count = 901
	Flags ([QuietMode2] ? ?): 0b0; Count = 901
	[i=15] UNK: 40; Count = 901
	[i=16] UNK: 00; Count = 901
	[i=17] UNK: 27; Count = 796
	[i=18] UNK: 00; Count = 901
	[i=19] UNK: 00; Count = 901
	[i=20] UNK: 01; Count = 901
	Flags ([CircPump] ? ? [Compressor]): 0b0; Count = 901
	Flags ([DHWmode] ? ?): 0b111; Count = 901
000A2034
	Count = 37
	Time = 0:6:33.362.0
	src = 0x00
	tgt = 0x0a
	type = 0x20
	Device identification request? (Thermostat); Count = 37
	[i=3] UNK: 34; Count = 37


0000100162021906110002000040000005400100000000
	Count = 81
	Time = 0:7:32.631.0
	src = 0x00
	tgt = 0x00
	type = 0x10
	Thermostat Settings (Thermostat -> Pump); Count = 901
	HeatingOn1: 01; Count = 901
	Gasmode Flags: 0b1100010; Count = 895
	Flags: (? [DHWon]): 02; Count = 895
	expected temp? = 25.6C; Count = 866
	Flags: ?: 0b10001; Count = 888
	[i=9] UNK: 00; Count = 901
	RoomTarget1: 02; Count = 901
	[i=11] UNK: 00; Count = 901
	i=12Flags ([QuietMode1] ? ?): 0b0; Count = 901
	i=13Flags ([QuietMode1] ? ?): 0b1000000; Count = 901
	[i=14] UNK: 00; Count = 901
	[i=15] UNK: 00; Count = 901
	Flags: ?: 0b101; Count = 94
	[i=17] UNK: 40; Count = 901
	[i=18] UNK: 01; Count = 901
	[i=19] UNK: 00; Count = 901
	Flags ([DHWactive] ? ? ? ? [DHWboosterActive] ?): 0b0; Count = 901
	DHWComfort1: 00; Count = 901
	[i=22] UNK: 00; Count = 901
40001001006202190611003A0000004000050000010007
	Count = 82
	Time = 0:7:32.699.0
	src = 0x40
	tgt = 0x00
	type = 0x10
	Thermostat Settings (Pump -> Thermostat); Coun

In [215]:
{x: all_decoded_lines[x] for x in all_decoded_lines.keys() if x.startswith("src = 0x00")}

{'src = 0x00tgt = 0x00type = 0x15Heartbeat (Thermostat -> Pump)': 226,
 'src = 0x00tgt = 0x00type = 0x10Thermostat Settings (Thermostat -> Pump)': 901,
 'src = 0x00tgt = 0x00type = 0x10HeatingOn1: 01': 901,
 'src = 0x00tgt = 0x00type = 0x10Gasmode Flags: 0b1100010': 895,
 'src = 0x00tgt = 0x00type = 0x10Flags: (? [DHWon]): 02': 895,
 'src = 0x00tgt = 0x00type = 0x10expected temp? = 25.6C': 866,
 'src = 0x00tgt = 0x00type = 0x10Flags: ?: 0b10001': 888,
 'src = 0x00tgt = 0x00type = 0x10[i=9] UNK: 00': 901,
 'src = 0x00tgt = 0x00type = 0x10RoomTarget1: 02': 901,
 'src = 0x00tgt = 0x00type = 0x10[i=11] UNK: 00': 901,
 'src = 0x00tgt = 0x00type = 0x10i=12Flags ([QuietMode1] ? ?): 0b0': 901,
 'src = 0x00tgt = 0x00type = 0x10i=13Flags ([QuietMode1] ? ?): 0b1000000': 901,
 'src = 0x00tgt = 0x00type = 0x10[i=14] UNK: 00': 901,
 'src = 0x00tgt = 0x00type = 0x10[i=15] UNK: 00': 901,
 'src = 0x00tgt = 0x00type = 0x10Flags: ?: 0b100111': 797,
 'src = 0x00tgt = 0x00type = 0x10[i=17] UNK: 40': 901,
 

In [140]:
decode_packet_data("0000100162021A01110002000040000027400100000000")

['src = 0x00',
 'tgt = 0x00',
 'type = 0x10',
 'Thermostat Settings',
 'HeatingOn1: 01',
 'GasMode: 62',
 'DHWon: 02',
 'expected temp? = 26.1C',
 '[i=8] UNK: 11',
 '[i=9] UNK: 00',
 'RoomTarget1: 02',
 '[i=11] UNK: 00',
 'Flags_Including_QuietMode1_at_pos2: 00',
 'Flags_Including_QuietMode1_at_pos2: 40',
 '[i=14] UNK: 00',
 '[i=15] UNK: 00',
 '[i=16] UNK: 27',
 '[i=17] UNK: 40',
 '[i=18] UNK: 01',
 '[i=19] UNK: 00',
 'Flags ([DHWactive] ? ? ? ? [DHWboosterActive] ?): 0b0',
 'DHWComfort1: 00',
 '[i=22] UNK: 00']

In [59]:
# Time of temp change from 79 to 78 deg, still auto
video_time_to_terminal_time(datetime.datetime(1,1,1, 0,0,30,275000))
# Time of temp change from 78 to 77 deg, still auto
video_time_to_terminal_time(datetime.datetime(1,1,1, 0,0,31,346000))
# --> 000011000000000019CD00000000
# -->  4000110000000000186000000000089C

datetime.datetime(1, 1, 1, 0, 6, 31, 739000)

In [144]:

# First observance of packet 0000100162021A01110002000040000027400100000000
# Note: At ~15s, temp changed from 78 to 79; Daikin controller takes ~6s to respond
#terminal_time_to_video_time(datetime.datetime(1,1,1, 0,6,22,824000))

# First observance of 24.5C (76.1F)
terminal_time_to_video_time(datetime.datetime(1,1,1, 0,6,40,546000))

datetime.datetime(1, 1, 1, 0, 0, 40, 153000)

In [154]:
len("40001001006202190611003A0000004000270000010007")

46